<a href="https://colab.research.google.com/github/tisch019/GraphenSNA/blob/main/RedditNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kpierter und angepaster Text aus dem Tutorial : https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

In [ ]:
import pandas as pd
import requests

In [ ]:
CLIENT_ID = 'imWZo6dEVJi6DQFj-GvUkQ'
SECRET_TOKEN = 'e6jNSjxu6qLD0MdUKzkwZZuxeQR_fA'

USERNAME = 'poliphemis'
PASSWORD = 'fg789Hzu4'

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': USERNAME,
        'password': PASSWORD}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'Hochschulprojekt zur Graphen SNA'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [ ]:
# Array of choosen reddits
reddits = ["python", "java", "php"]

# Dictionary of API-responses
responses = {}
# get hot posts of choosen reddits
for reddit in reddits:
  responses[reddit] = requests.get("https://oauth.reddit.com/r/"+reddit+"/hot",
                   headers=headers, params={'limit':'20', 't':'month'})

In [ ]:
# initialize dictionary of dataframes
dataframes = {}
# add all responses to dictionary
for reddit in reddits:
  dataframes[reddit] = pd.DataFrame()  # initialize dataframe
  # loop through each post retrieved from GET request
  for post in responses[reddit].json()['data']['children']:
      # append relevant data to dataframe
      dataframes[reddit] = dataframes[reddit].append({
          'subreddit': post['data']['subreddit'],
          'author': post['data']['author'],
          'title': post['data']['title'],
          'selftext': post['data']['selftext'],
          'upvote_ratio': post['data']['upvote_ratio'],
          'ups': post['data']['ups'],
          'downs': post['data']['downs'],
          'score': post['data']['score']
      }, ignore_index=True)

In [ ]:
for reddit in reddits:
  print(dataframes[reddit])

                 author  downs  ...    ups upvote_ratio
0            Im__Joseph    0.0  ...   12.0         0.94
1            Im__Joseph    0.0  ...    0.0         0.43
2        n3buchadnezzar    0.0  ...   49.0         0.86
3         Quantumlyhigh    0.0  ...   63.0         0.89
4             vapen_hem    0.0  ...  308.0         0.90
5            Fanrounder    0.0  ...   33.0         0.76
6            SudoLogeek    0.0  ...    6.0         0.88
7              hobbesid    0.0  ...    4.0         0.71
8       These-Ease-4410    0.0  ...    6.0         0.80
9          ellsphillips    0.0  ...    5.0         0.86
10             EmmaGao8    0.0  ...    8.0         0.75
11  ConstructionHot6883    0.0  ...    1.0         1.00
12        LinkusThinkus    0.0  ...   98.0         0.92
13       sepandhaghighi    0.0  ...   47.0         0.92
14            rszamszur    0.0  ...   20.0         0.82
15            AfroBoyUg    0.0  ...    1.0         0.57
16            Feitgemel    0.0  ...    3.0      

In [ ]:
# Alle Keywörter die bei einem Post abgefragt werden können

post['data'].keys()


dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html', 'likes', 'suggested_sort', 'banned_at_u

In [ ]:
#reorganising the data as a dictionary with authors (=future nodes in network) as keys
author_dict = {}

#HIER IN DER REGEL AUSKOMMENTIEREN
number_of_nodes = len(reddits)

#TODO ups beziehen sich bei mehreren Posts bislang nur auf den ersten Post in der Schleife!
for key in dataframes:
    if dataframes[key]['author'] in author_dict:
      author_dict[dataframes[key]['author']['subreddit'].append(row.get('subreddit'))
    else:
        number_of_nodes += 1
        author_dict[row.get('Autor')] = {
         'author_fullname': row.get('Autor_fullname'),
         'subreddit': [row.get('subreddit')],
         'ups': row.get('ups'),
         'node_id': number_of_nodes
        }

author_dict

SyntaxError: ignored

In [ ]:
def make_vertex_in_pajek(count_vertices, name, shape, color):
  return '\n {} "{}" {} ic {} bc {}'.format(count_vertices, name, shape, color, color)

def make_arc_in_pajek(from_vertex, to_vertex, weight=1):
  return ' {} {} {}\n'.format(from_vertex, to_vertex, weight)

pajek_vertices = "*Vertices {}".format(len(reddits)+len(author_dict.keys()))
pajek_arcs = '*Arcs :1 "Authors in subreddits"\n'

counter = 1
#make vertices for subreddits
for subreddit in reddits:
  pajek_vertices = pajek_vertices + make_vertex_in_pajek(counter, subreddit, 'box', 'Cornflowerblue')
  counter += 1

#make vertices for authors and add arcs
for author in author_dict:
  pajek_vertices = pajek_vertices + make_vertex_in_pajek(author_dict[author]['node_id'], author, 'ellipse', 'Melon')

#add one arc for each subreddit the author has a top post in
i = 1

#PLEASE ADAPT THESE LINES IF THE reddits-ARRAY IS NOT CORRECTLY USED!!!
for subreddit in reddits:
  num = author_dict[author]['subreddit'].count(subreddit)
  if num > 0:
    pajek_arcs = pajek_arcs + make_arc_in_pajek(author_dict[author]['node_id'], i, num)
  i += 1

print(pajek_vertices)
print(pajek_arcs)